In [ ]:
# ========================================
# 1) Connect to Google Drive
# ========================================
from google.colab import drive
drive.mount('/content/drive')

#MODEL_DIR = "/content/drive/MyDrive/deep_learning"


Mounted at /content/drive


In [ ]:
# ========================================
# 2) Install required libraries
# ========================================
!pip install -U transformers datasets accelerate evaluate optuna wandb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install evaluate

In [ ]:
!pip install -U transformers


In [ ]:
# ========================================
# 3) Import libraries
# ========================================
import os, time, copy, tempfile, inspect
import numpy as np
import pandas as pd
import torch, torch.nn as nn
from torch.quantization import quantize_dynamic
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
import os
import re
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, DataCollatorWithPadding
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report


In [ ]:
# Check if GPU is available and select device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# === Load data ===
#test_df = pd.read_csv('/content/drive/MyDrive/deep_learning/val_processed.csv', encoding='latin1')
from pathlib import Path
import pandas as pd

# Navigate from model folder to data folder
current_dir = Path.cwd()  # models/bert-base-uncased/
models_dir = current_dir.parent  # models/
project_root = models_dir.parent  # project root
data_dir = models_dir / 'data'

# Load data
test_df = pd.read_csv(data_dir / 'test_processed.csv', encoding='latin1')

print(f"Test shape: {test_df.shape}")

# --------- config ----------
MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment-latest"
#MODEL_DIR  = "/content/drive/MyDrive/deep_learning"
MODEL_DIR  =data_dir

In [ ]:
test_df["label"] = test_df["Sentiment"].map(label2id)

In [ ]:
def clean_for_cardiffnlp(text):
    if pd.isnull(text):
        return ""

    tokens = []
    for t in text.split(" "):
        if t.startswith("@") and len(t) > 1:
            tokens.append("@user")
        elif t.startswith("http"):
            tokens.append("http")
        else:
            tokens.append(t)
    text = " ".join(tokens)

    # Normalize common COVID variants to "covid"
    text = re.sub(r"\b(coronaviruspandemic|covid[_\s-]*2019|covid[_\s-]*19|covid2019|coronavirus2019|coronavirus|corona)\b", "covid", text, flags=re.IGNORECASE)

    # Decode HTML entities
    text = html.unescape(text)

    # Normalize whitespace and repeated punctuation (optional)
    text = re.sub(r"\s+", " ", text).strip()
    text = re.sub(r"(\.\s*){2,}", ". ", text)
    text = re.sub(r"([!?]){2,}", r"\1", text)
    text = re.sub(r"(\?\s+){2,}", "?", text)
    text = re.sub(r"(\!\s+){2,}", "!", text)

    return text


In [ ]:
BASE_CHECKPOINT = "cardiffnlp/twitter-roberta-base-sentiment-latest"
LABEL_COL = "label"
BATCH_SIZE = 128
NUM_WORKERS = 2
PIN_MEMORY = True


# ==== tokenizer + collator ====
tokenizer = AutoTokenizer.from_pretrained(BASE_CHECKPOINT, use_fast=True)
collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ==== Class Dataset ====
class TextClsDataset(Dataset):
    def __init__(self, df, text_col, label_col):
        self.texts = df[text_col].astype(str).tolist()
        self.labels = df[label_col].astype(int).tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = tokenizer(
            self.texts[idx],
            truncation=True,
            max_length=128,
            return_tensors=None
        )
        enc["labels"] = self.labels[idx]
        return enc

# prepere datasets
test_ds_clean = TextClsDataset(test_df, text_col="ProcessedTweet",        label_col=LABEL_COL)

# DataLoaders
test_loader_clean = DataLoader(
    test_ds_clean,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    collate_fn=collator
)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
# ============ CUSTOM ROBERTA WITH DROPOUT ============
class RobertaWithDropout(nn.Module):
    """
    Custom RoBERTa model with configurable dropout
    """
    def __init__(self, model_name, num_labels, dropout_rate=0.1):
        super(RobertaWithDropout, self).__init__()

        self.roberta = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.roberta.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return type('obj', (object,), {'logits': logits})()


In [ ]:
#MODEL_DIR = "/content/drive/MyDrive/deep_learning"

# Map -> preprocessing
MODEL_PREPROC = {
    "best_model2_YR.pt": "clean",
}

# Eval Function
def evaluate_model(model, dataloader, device):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Report
    print("Classification Report:")
    print(classification_report(
        all_labels, all_preds,
        target_names=['Extremely Negative', 'Negative', 'Neutral', 'Positive', 'Extremely Positive']

    ))

    return {
        "accuracy": accuracy_score(all_labels, all_preds),
        "f1": f1_score(all_labels, all_preds, average='macro'),
        "precision": precision_score(all_labels, all_preds, average='macro'),
        "recall": recall_score(all_labels, all_preds, average='macro')
    }

# Eval All Models
def evaluate_all_models(model_dir, model_preproc_map, test_loader_clean, device):
    results = {}

    for model_file, preproc_type in model_preproc_map.items():
        print(f"\n=== Evaluating {model_file} ({preproc_type}) ===")

        model_path = os.path.join(model_dir, model_file)
        model = RobertaWithDropout(
        "cardiffnlp/twitter-roberta-base-sentiment-latest",
        num_labels=5,
        dropout_rate=0.2)
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.to(device)

        if preproc_type == "clean":
            test_loader = test_loader_clean
        elif preproc_type == "light":
            test_loader = test_loader_light
        else:
            raise ValueError(f"Unknown preprocessing type: {preproc_type}")


        metrics = evaluate_model(model, test_loader, device)
        results[model_file] = metrics


    print("\n=== Summary of All Models ===")
    for model_file, metrics in results.items():
        print(f"{model_file}: "
              f"Acc={metrics['accuracy']:.4f}, "
              f"F1={metrics['f1']:.4f}, "
              f"Precision={metrics['precision']:.4f}, "
              f"Recall={metrics['recall']:.4f}")

    return results


In [ ]:
results = evaluate_all_models(MODEL_DIR, MODEL_PREPROC, test_loader_clean, device)


=== Evaluating best_model2_YR.pt (clean) ===


pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Classification Report:
                    precision    recall  f1-score   support

Extremely Negative       0.85      0.89      0.87       592
          Negative       0.82      0.84      0.83      1041
           Neutral       0.87      0.83      0.85       619
          Positive       0.80      0.84      0.82       947
Extremely Positive       0.92      0.82      0.86       599

          accuracy                           0.84      3798
         macro avg       0.85      0.84      0.85      3798
      weighted avg       0.84      0.84      0.84      3798


=== Summary of All Models ===
best_model2_YR.pt: Acc=0.8415, F1=0.8460, Precision=0.8516, Recall=0.8421


# Post-training Dynamic Quantization

In [ ]:
DEVICE = torch.device("cpu")
# Check
print(f"Using device: {DEVICE}")

Using device: cpu


In [ ]:
# ======================
# Step 1: Robust loader for  Roberta
# ======================

import os
import copy
import tempfile
from pathlib import Path

import torch
import torch.nn as nn
from torch.quantization import quantize_dynamic

from transformers import AutoTokenizer, DataCollatorWithPadding
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

# ======================
# Basic configuration
# ======================
MODEL_DIR  =data_dir
MODEL_FILE  = "best_model2_YR.pt"
BACKBONE    = "cardiffnlp/twitter-roberta-base-sentiment-latest"
NUM_LABELS  = 5
DROPOUT     = 0.2
OUT_DIR     = os.path.join(MODEL_DIR, "quantized_int8")
os.makedirs(OUT_DIR, exist_ok=True)

ordered_labels = ["Extremely Negative","Negative","Neutral","Positive","Extremely Positive"]

def _state_dict_size_mb(model_or_sd) -> float:
    sd = model_or_sd if isinstance(model_or_sd, dict) else model_or_sd.state_dict()
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pt") as tmp:
        torch.save(sd, tmp.name)
        size_mb = os.path.getsize(tmp.name) / (1024**2)
    os.remove(tmp.name)
    return size_mb

# ======================
# 1) Load FP32 model
# ======================
model_path = os.path.join(MODEL_DIR, MODEL_FILE)
model = RobertaWithDropout(BACKBONE, num_labels=NUM_LABELS, dropout_rate=DROPOUT)
state_dict = torch.load(model_path, map_location="cpu")
model.load_state_dict(state_dict, strict=True)
model.eval().cpu()

param_count = sum(p.numel() for p in model.parameters())
size_fp32   = _state_dict_size_mb(model)
print(f"Loaded {MODEL_FILE} | params={param_count:,} | state_dict={size_fp32:.2f} MB")

# ======================
# 2) Dynamic quantization on encoder only (INT8)
# ======================
q_model = copy.deepcopy(model).eval().cpu()
assert hasattr(q_model, "roberta") and hasattr(q_model.roberta, "encoder"), "Expected .roberta.encoder to exist"
q_model.roberta.encoder = quantize_dynamic(q_model.roberta.encoder, {nn.Linear}, dtype=torch.qint8)

size_int8 = _state_dict_size_mb(q_model)
print(f"State dict size: FP32={size_fp32:.2f} MB -> INT8={size_int8:.2f} MB")

# ======================
# 3) Save quantized state_dict
# ======================
save_name  = f"{Path(MODEL_FILE).stem}_quantized_int8_sd.pt"
quant_path = os.path.join(OUT_DIR, save_name)
torch.save(q_model.state_dict(), quant_path)
print(f"Quantized model saved to: {quant_path}")



Loaded best_model2_YR.pt | params=124,649,477 | state_dict=475.58 MB
State dict size: FP32=475.58 MB -> INT8=232.63 MB
Quantized model saved to: /content/drive/MyDrive/deep_learning/quantized_int8/best_model2_YR_quantized_int8_sd.pt


In [ ]:
# ======================
# 4) Optional evaluation on CPU (runs only if `test_df` exists)
# ======================
@torch.inference_mode()
def eval_int8_cpu(model, dataloader, label_names=None):
    """Evaluate model on CPU and print a per-class report."""
    model.eval()
    model.cpu()
    y_true, y_pred = [], []

    for batch in dataloader:
        labels = batch.pop("labels")
        # Ensure tensors
        batch = {k: (v if isinstance(v, torch.Tensor) else torch.tensor(v)) for k, v in batch.items()}
        outputs = model(**batch)

        # Robust logits extraction
        if hasattr(outputs, "logits"):
            logits = outputs.logits
        elif isinstance(outputs, dict) and "logits" in outputs:
            logits = outputs["logits"]
        elif isinstance(outputs, (list, tuple)):
            logits = outputs[0]
        else:
            logits = outputs  # if the model returns logits directly

        preds = torch.argmax(logits, dim=-1)
        y_true.extend(labels.cpu().tolist())
        y_pred.extend(preds.cpu().tolist())

    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=label_names, digits=4))

    metrics = {
        "accuracy":  accuracy_score(y_true, y_pred),
        "f1":        f1_score(y_true, y_pred, average="macro"),
        "precision": precision_score(y_true, y_pred, average="macro", zero_division=0),
        "recall":    recall_score(y_true, y_pred, average="macro"),
    }
    print(
        f"Accuracy={metrics['accuracy']:.4f} | F1(macro)={metrics['f1']:.4f} | "
        f"Precision(macro)={metrics['precision']:.4f} | Recall(macro)={metrics['recall']:.4f}"
    )
    return metrics


class _EvalRobertaDS(Dataset):
    """Simple dataset for preprocessed texts and integer labels."""
    def __init__(self, texts, labels, max_len=128, backbone=BACKBONE):
        self.texts = list(texts)
        self.labels = list(labels)
        self.max_len = max_len
        self.tok = AutoTokenizer.from_pretrained(backbone, use_fast=True)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, i):
        enc = self.tok(self.texts[i], truncation=True, max_length=self.max_len)
        enc["labels"] = int(self.labels[i])
        return enc

# === 4) Evaluate with your existing loader ===
print("Evaluating INT8 with existing test_loader_clean...")
int8_metrics = eval_int8_cpu(q_model, test_loader_clean, label_names=ordered_labels)


Evaluating INT8 with existing test_loader_clean...
Classification Report:
                    precision    recall  f1-score   support

Extremely Negative     0.8769    0.8547    0.8657       592
          Negative     0.8183    0.8348    0.8264      1041
           Neutral     0.8718    0.8352    0.8531       619
          Positive     0.7555    0.8648    0.8065       947
Extremely Positive     0.9253    0.7446    0.8252       599

          accuracy                         0.8312      3798
         macro avg     0.8496    0.8268    0.8354      3798
      weighted avg     0.8374    0.8312    0.8317      3798

Accuracy=0.8312 | F1(macro)=0.8354 | Precision(macro)=0.8496 | Recall(macro)=0.8268


# unstructured pruning

In [ ]:
# prune_best_model2_roberta.py
import os
import tempfile
from pathlib import Path

import torch
import torch.nn as nn
from torch.nn.utils import prune
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

# If RobertaWithDropout is defined elsewhere, import it:
# from your_model_defs import RobertaWithDropout

# ======================
# Basic configuration
# ======================
MODEL_DIR  =data_dir
MODEL_FILE = "best_model2_YR.pt"
BACKBONE   = "cardiffnlp/twitter-roberta-base-sentiment-latest"
NUM_LABELS = 5
DROPOUT    = 0.2

PRUNE_AMOUNT         = 0.40  # 40% global unstructured L1
PRUNE_CLASSIFIER     = False # keep classifier dense by default
PRUNED_SD_SAVE_PATH  = os.path.join(MODEL_DIR, f"{Path(MODEL_FILE).stem}_pruned40_sd.pt")

# Use your existing labels or override here
ordered_labels = ["Extremely Negative","Negative","Neutral","Positive","Extremely Positive"]

# ======================
# Utilities
# ======================
def state_dict_size_mb(model_or_sd) -> float:
    sd = model_or_sd if isinstance(model_or_sd, dict) else model_or_sd.state_dict()
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pt") as tmp:
        torch.save(sd, tmp.name)
        size_mb = os.path.getsize(tmp.name) / (1024**2)
    os.remove(tmp.name)
    return size_mb

def load_fp32_model_state(model_dir, model_file):
    """Load FP32 weights into the exact architecture wrapper."""
    path = os.path.join(model_dir, model_file)
    model = RobertaWithDropout(BACKBONE, num_labels=NUM_LABELS, dropout_rate=DROPOUT)
    state = torch.load(path, map_location="cpu")
    model.load_state_dict(state, strict=True)
    model.eval().cpu()
    return model

def collect_encoder_linear_params(model, prune_classifier=False):
    """
    Collect (module, 'weight') pairs for nn.Linear layers.
    By default, only prune Linear layers under 'roberta.encoder'.
    If prune_classifier=True, also include classifier Linear layers.
    """
    targets = []
    for name, module in model.named_modules():
        if not isinstance(module, nn.Linear):
            continue
        is_encoder = name.startswith("roberta.encoder")
        is_classifier = name.startswith("classifier") or ".classifier." in name
        if is_encoder:
            targets.append((module, "weight"))
        elif prune_classifier and is_classifier:
            targets.append((module, "weight"))
    return targets

def apply_global_l1(model, amount=0.4, prune_classifier=False):
    """Apply global unstructured L1 pruning over selected Linear weights."""
    targets = collect_encoder_linear_params(model, prune_classifier=prune_classifier)
    if len(targets) == 0:
        raise RuntimeError("No Linear layers collected for pruning. Check your module naming/paths.")
    prune.global_unstructured(targets, pruning_method=prune.L1Unstructured, amount=amount)

    # Report density/sparsity on masked weights (viewed with the pruning mask)
    total, nonzero = 0, 0
    for m, _ in targets:
        w = m.weight  # masked view
        total   += w.numel()
        nonzero += (w != 0).sum().item()
    density  = nonzero / total if total else 1.0
    sparsity = 1.0 - density
    print(f"Applied global L1 pruning: amount={amount:.2f} | density={density:.2%} | sparsity={sparsity:.2%}")
    return targets, density, sparsity

def remove_pruning_masks(targets):
    """Make pruning permanent by removing mask reparameterizations."""
    for m, _ in targets:
        prune.remove(m, "weight")

@torch.inference_mode()
def evaluate_model_cpu(model, dataloader, label_names=None):
    """Generic CPU evaluation for classification models that return logits."""
    model.eval().cpu()
    y_true, y_pred = [], []
    for batch in dataloader:
        # Ensure tensors are on CPU
        for k, v in list(batch.items()):
            if isinstance(v, torch.Tensor):
                batch[k] = v.cpu()

        labels = batch.pop("labels") if "labels" in batch else batch.pop("label", None)
        outputs = model(**batch)

        # Robust logits extraction
        if hasattr(outputs, "logits"):
            logits = outputs.logits
        elif isinstance(outputs, dict) and "logits" in outputs:
            logits = outputs["logits"]
        elif isinstance(outputs, (list, tuple)):
            logits = outputs[0]
        else:
            logits = outputs

        preds = torch.argmax(logits, dim=-1)
        y_pred.extend(preds.cpu().tolist())
        if labels is not None:
            y_true.extend(labels.cpu().tolist())

    if len(y_true) > 0:
        print("Classification Report:")
        print(classification_report(y_true, y_pred, target_names=label_names, digits=4))
        metrics = {
            "accuracy":  accuracy_score(y_true, y_pred),
            "f1":        f1_score(y_true, y_pred, average="macro"),
            "precision": precision_score(y_true, y_pred, average="macro", zero_division=0),
            "recall":    recall_score(y_true, y_pred, average="macro"),
        }
        print(
            f"Accuracy={metrics['accuracy']:.4f} | F1(macro)={metrics['f1']:.4f} | "
            f"Precision(macro)={metrics['precision']:.4f} | Recall(macro)={metrics['recall']:.4f}"
        )
        return metrics
    else:
        print("Labels not found in dataloader batches; returned predictions only.")
        return {}

# ======================
# Run pruning workflow
# ======================
model = load_fp32_model_state(MODEL_DIR, MODEL_FILE)
print(f"Baseline state_dict size (dense): {state_dict_size_mb(model):.2f} MB")

targets, density, sparsity = apply_global_l1(
    model,
    amount=PRUNE_AMOUNT,
    prune_classifier=PRUNE_CLASSIFIER
)

# (Optional) short fine-tuning step to recover accuracy could be placed here.

remove_pruning_masks(targets)
print(f"Pruned state_dict size (dense save): {state_dict_size_mb(model):.2f} MB")

torch.save(model.state_dict(), PRUNED_SD_SAVE_PATH)
print("Saved pruned state_dict to:", PRUNED_SD_SAVE_PATH)

# ======================
# Evaluate with your existing loader
# ======================
# Assumes you already have: test_loader_clean
# If your loader is named differently, replace below.
if "test_loader_clean" in globals():
    print("Evaluating PRUNED model on test_loader_clean (CPU)...")
    pruned_metrics = evaluate_model_cpu(model, test_loader_clean, label_names=ordered_labels)
else:
    print("Skipping evaluation: `test_loader_clean` not found in globals().")


Baseline state_dict size (dense): 475.58 MB
Applied global L1 pruning: amount=0.40 | density=60.00% | sparsity=40.00%
Pruned state_dict size (dense save): 475.58 MB
Saved pruned state_dict to: /content/drive/MyDrive/deep_learning/best_model2_YR_pruned40_sd.pt
Evaluating PRUNED model on test_loader_clean (CPU)...
Classification Report:
                    precision    recall  f1-score   support

Extremely Negative     0.9682    0.4628    0.6263       592
          Negative     0.6879    0.8809    0.7725      1041
           Neutral     0.8592    0.8675    0.8633       619
          Positive     0.6327    0.8986    0.7426       947
Extremely Positive     1.0000    0.3539    0.5228       599

          accuracy                         0.7349      3798
         macro avg     0.8296    0.6928    0.7055      3798
      weighted avg     0.7950    0.7349    0.7177      3798

Accuracy=0.7349 | F1(macro)=0.7055 | Precision(macro)=0.8296 | Recall(macro)=0.6928


In [ ]:
# prune_best_model2_roberta.py
import os
import tempfile
from pathlib import Path

import torch
import torch.nn as nn
from torch.nn.utils import prune
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

# If RobertaWithDropout is defined elsewhere, import it:
# from your_model_defs import RobertaWithDropout

# ======================
# Basic configuration
# ======================
MODEL_DIR  =data_dir
MODEL_FILE = "best_model2_YR.pt"
BACKBONE   = "cardiffnlp/twitter-roberta-base-sentiment-latest"
NUM_LABELS = 5
DROPOUT    = 0.2

PRUNE_AMOUNT         = 0.30  # 30% global unstructured L1
PRUNE_CLASSIFIER     = False # keep classifier dense by default
PRUNED_SD_SAVE_PATH  = os.path.join(MODEL_DIR, f"{Path(MODEL_FILE).stem}_pruned30_sd.pt")

# Use your existing labels or override here
ordered_labels = ["Extremely Negative","Negative","Neutral","Positive","Extremely Positive"]

# ======================
# Utilities
# ======================
def state_dict_size_mb(model_or_sd) -> float:
    sd = model_or_sd if isinstance(model_or_sd, dict) else model_or_sd.state_dict()
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pt") as tmp:
        torch.save(sd, tmp.name)
        size_mb = os.path.getsize(tmp.name) / (1024**2)
    os.remove(tmp.name)
    return size_mb

def load_fp32_model_state(model_dir, model_file):
    """Load FP32 weights into the exact architecture wrapper."""
    path = os.path.join(model_dir, model_file)
    model = RobertaWithDropout(BACKBONE, num_labels=NUM_LABELS, dropout_rate=DROPOUT)
    state = torch.load(path, map_location="cpu")
    model.load_state_dict(state, strict=True)
    model.eval().cpu()
    return model

def collect_encoder_linear_params(model, prune_classifier=False):
    """
    Collect (module, 'weight') pairs for nn.Linear layers.
    By default, only prune Linear layers under 'roberta.encoder'.
    If prune_classifier=True, also include classifier Linear layers.
    """
    targets = []
    for name, module in model.named_modules():
        if not isinstance(module, nn.Linear):
            continue
        is_encoder = name.startswith("roberta.encoder")
        is_classifier = name.startswith("classifier") or ".classifier." in name
        if is_encoder:
            targets.append((module, "weight"))
        elif prune_classifier and is_classifier:
            targets.append((module, "weight"))
    return targets

def apply_global_l1(model, amount=0.4, prune_classifier=False):
    """Apply global unstructured L1 pruning over selected Linear weights."""
    targets = collect_encoder_linear_params(model, prune_classifier=prune_classifier)
    if len(targets) == 0:
        raise RuntimeError("No Linear layers collected for pruning. Check your module naming/paths.")
    prune.global_unstructured(targets, pruning_method=prune.L1Unstructured, amount=amount)

    # Report density/sparsity on masked weights (viewed with the pruning mask)
    total, nonzero = 0, 0
    for m, _ in targets:
        w = m.weight  # masked view
        total   += w.numel()
        nonzero += (w != 0).sum().item()
    density  = nonzero / total if total else 1.0
    sparsity = 1.0 - density
    print(f"Applied global L1 pruning: amount={amount:.2f} | density={density:.2%} | sparsity={sparsity:.2%}")
    return targets, density, sparsity

def remove_pruning_masks(targets):
    """Make pruning permanent by removing mask reparameterizations."""
    for m, _ in targets:
        prune.remove(m, "weight")

@torch.inference_mode()
def evaluate_model_cpu(model, dataloader, label_names=None):
    """Generic CPU evaluation for classification models that return logits."""
    model.eval().cpu()
    y_true, y_pred = [], []
    for batch in dataloader:
        # Ensure tensors are on CPU
        for k, v in list(batch.items()):
            if isinstance(v, torch.Tensor):
                batch[k] = v.cpu()

        labels = batch.pop("labels") if "labels" in batch else batch.pop("label", None)
        outputs = model(**batch)

        # Robust logits extraction
        if hasattr(outputs, "logits"):
            logits = outputs.logits
        elif isinstance(outputs, dict) and "logits" in outputs:
            logits = outputs["logits"]
        elif isinstance(outputs, (list, tuple)):
            logits = outputs[0]
        else:
            logits = outputs

        preds = torch.argmax(logits, dim=-1)
        y_pred.extend(preds.cpu().tolist())
        if labels is not None:
            y_true.extend(labels.cpu().tolist())

    if len(y_true) > 0:
        print("Classification Report:")
        print(classification_report(y_true, y_pred, target_names=label_names, digits=4))
        metrics = {
            "accuracy":  accuracy_score(y_true, y_pred),
            "f1":        f1_score(y_true, y_pred, average="macro"),
            "precision": precision_score(y_true, y_pred, average="macro", zero_division=0),
            "recall":    recall_score(y_true, y_pred, average="macro"),
        }
        print(
            f"Accuracy={metrics['accuracy']:.4f} | F1(macro)={metrics['f1']:.4f} | "
            f"Precision(macro)={metrics['precision']:.4f} | Recall(macro)={metrics['recall']:.4f}"
        )
        return metrics
    else:
        print("Labels not found in dataloader batches; returned predictions only.")
        return {}

# ======================
# Run pruning workflow
# ======================
model = load_fp32_model_state(MODEL_DIR, MODEL_FILE)
print(f"Baseline state_dict size (dense): {state_dict_size_mb(model):.2f} MB")

targets, density, sparsity = apply_global_l1(
    model,
    amount=PRUNE_AMOUNT,
    prune_classifier=PRUNE_CLASSIFIER
)

# (Optional) short fine-tuning step to recover accuracy could be placed here.

remove_pruning_masks(targets)
print(f"Pruned state_dict size (dense save): {state_dict_size_mb(model):.2f} MB")

torch.save(model.state_dict(), PRUNED_SD_SAVE_PATH)
print("Saved pruned state_dict to:", PRUNED_SD_SAVE_PATH)

# ======================
# Evaluate
# ======================
# Assumes you already have: test_loader_clean
# If your loader is named differently, replace below.
if "test_loader_clean" in globals():
    print("Evaluating PRUNED model on test_loader_clean (CPU)...")
    pruned_metrics = evaluate_model_cpu(model, test_loader_clean, label_names=ordered_labels)
else:
    print("Skipping evaluation: `test_loader_clean` not found in globals().")


Baseline state_dict size (dense): 475.58 MB
Applied global L1 pruning: amount=0.30 | density=70.00% | sparsity=30.00%
Pruned state_dict size (dense save): 475.58 MB
Saved pruned state_dict to: /content/drive/MyDrive/deep_learning/best_model2_YR_pruned40_sd.pt
Evaluating PRUNED model on test_loader_clean (CPU)...
Classification Report:
                    precision    recall  f1-score   support

Extremely Negative     0.9257    0.7787    0.8459       592
          Negative     0.7983    0.8780    0.8362      1041
           Neutral     0.8576    0.8368    0.8471       619
          Positive     0.7389    0.8817    0.8040       947
Extremely Positive     0.9715    0.6828    0.8020       599

          accuracy                         0.8260      3798
         macro avg     0.8584    0.8116    0.8270      3798
      weighted avg     0.8403    0.8260    0.8261      3798

Accuracy=0.8260 | F1(macro)=0.8270 | Precision(macro)=0.8584 | Recall(macro)=0.8116


# Knowledge Distillation Demo

In [ ]:
# 0) Assumes you already defined:
# - train_df, eval_df, test_df
# - ordered_labels, label2id/id2label
# - tokenizer = AutoTokenizer.from_pretrained(BASE_CHECKPOINT, use_fast=True)
# - collator = DataCollatorWithPadding(tokenizer=tokenizer)
# - TextClsDataset class (returns dict with input_ids/attention_mask/labels)
# - test_loader_clean  (for final test only)

NUM_LABELS = len(ordered_labels)

# 1) Build train/eval datasets for Trainer (NOT loaders)
train_ds = TextClsDataset(train_df, text_col="ProcessedTweet", label_col="label")
val_ds   = TextClsDataset(eval_df,   text_col="ProcessedTweet", label_col="label")

# 2) Teacher (RoBERTa) – frozen
BACKBONE_TEACHER = "cardiffnlp/twitter-roberta-base-sentiment-latest"
TEACHER_WEIGHTS  = "/content/drive/MyDrive/deep_learning/best_model2_YR.pt"

def load_fp32_teacher_roberta(path, num_labels):
    model = RobertaWithDropout(BACKBONE_TEACHER, num_labels=num_labels, dropout_rate=0.2)
    state = torch.load(path, map_location="cpu")
    model.load_state_dict(state, strict=True)
    model.eval()
    for p in model.parameters():
        p.requires_grad = False
    return model

device  = "cuda" if torch.cuda.is_available() else "cpu"
teacher = load_fp32_teacher_roberta(TEACHER_WEIGHTS, NUM_LABELS).to(device)

# 3) Student = DistilRoBERTa
from transformers import AutoModelForSequenceClassification
student = AutoModelForSequenceClassification.from_pretrained(
    "distilroberta-base", num_labels=NUM_LABELS
).to(device)

# 4) KD Trainer
from transformers import Trainer, TrainingArguments
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

class DistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, temperature=2.0, alpha=0.7, **kwargs):
        super().__init__(*args, **kwargs)
        assert teacher_model is not None, "teacher_model must be provided"
        self.teacher = teacher_model
        self.temperature = temperature
        self.alpha = alpha

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Ensure RoBERTa-compatible inputs
        inputs = dict(inputs)
        inputs.pop("token_type_ids", None)

        # Forward (student) — Trainer already moves to device
        outputs_student = model(**inputs)
        logits_student = outputs_student.logits

        # Forward (teacher) — frozen, same device as student
        with torch.no_grad():
            t_inputs = {
                "input_ids": inputs["input_ids"],
                "attention_mask": inputs.get("attention_mask", None),
            }
            # some training loops add other keys; ignore safely
            t_outputs = self.teacher(**t_inputs)
            logits_teacher = t_outputs["logits"] if isinstance(t_outputs, dict) else t_outputs.logits

        # Hard labels CE
        loss_ce = F.cross_entropy(logits_student, inputs["labels"])

        # KD loss (KL) with temperature
        T = self.temperature
        loss_kl = F.kl_div(
            F.log_softmax(logits_student / T, dim=-1),
            F.softmax(logits_teacher / T, dim=-1),
            reduction="batchmean"
        ) * (T ** 2)

        loss = self.alpha * loss_ce + (1.0 - self.alpha) * loss_kl
        return (loss, outputs_student) if return_outputs else loss


def compute_metrics_fn(eval_pred):
    # supports both EvalPrediction object and (preds, labels) tuple
    preds = getattr(eval_pred, "predictions", eval_pred[0])
    labels = getattr(eval_pred, "label_ids", eval_pred[1])
    if isinstance(preds, tuple):  # some models return (logits,)
        preds = preds[0]
    preds = np.argmax(preds, axis=-1)
    return {
        "accuracy":  accuracy_score(labels, preds),
        "f1_macro":  f1_score(labels, preds, average="macro"),
        "precision": precision_score(labels, preds, average="macro", zero_division=0),
        "recall":    recall_score(labels, preds, average="macro"),
    }

# 5) TrainingArguments — minimal, to avoid version issues with older transformers
training_args = TrainingArguments(
    output_dir="./distill_results_roberta_student",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    fp16=torch.cuda.is_available(),
    report_to="none",
    remove_unused_columns=False,   # keep our dict keys intact
)

trainer_distill = DistillationTrainer(
    model=student,
    teacher_model=teacher,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,           # validation during/after training
    compute_metrics=compute_metrics_fn,
    data_collator=collator,        # same collator you used for loaders
)

# 6) Train + validation metrics
trainer_distill.train()
val_metrics = trainer_distill.evaluate()
print("Validation metrics:", val_metrics)

# 7) Final test with your existing test_loader_clean
@torch.inference_mode()
def evaluate_model_cpu(model, dataloader, label_names=None):
    model.eval().cpu()
    y_true, y_pred = [], []
    for batch in dataloader:
        for k, v in list(batch.items()):
            if isinstance(v, torch.Tensor): batch[k] = v.cpu()
        labels = batch.pop("labels")
        out = model(**batch)
        logits = out.logits if hasattr(out, "logits") else (out["logits"] if isinstance(out, dict) else out[0])
        y_pred.extend(torch.argmax(logits, dim=-1).cpu().tolist())
        y_true.extend(labels.cpu().tolist())
    print("Test Accuracy/F1/Precision/Recall (macro):",
          accuracy_score(y_true, y_pred),
          f1_score(y_true, y_pred, average="macro"),
          precision_score(y_true, y_pred, average="macro", zero_division=0),
          recall_score(y_true, y_pred, average="macro"))
    return {
        "accuracy":  accuracy_score(y_true, y_pred),
        "f1_macro":  f1_score(y_true, y_pred, average="macro"),
        "precision": precision_score(y_true, y_pred, average="macro", zero_division=0),
        "recall":    recall_score(y_true, y_pred, average="macro"),
    }

test_metrics = evaluate_model_cpu(trainer_distill.model, test_loader_clean, label_names=ordered_labels)
print("Test metrics:", test_metrics)

# 8) Save student
trainer_distill.save_model("./distill_results_roberta_student/best_student")


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
50,2.580400
100,2.077100
150,1.881700
200,1.681500
250,1.502000
300,1.496100
350,1.484300
400,1.415100
450,1.319500
500,1.310400


Validation metrics: {'eval_loss': 0.7392584681510925, 'eval_accuracy': 0.8440233236151603, 'eval_f1_macro': 0.8474009621720165, 'eval_precision': 0.8426385402623053, 'eval_recall': 0.8564314078474611, 'eval_runtime': 18.5705, 'eval_samples_per_second': 443.283, 'eval_steps_per_second': 6.946, 'epoch': 5.0}
Test Accuracy/F1/Precision/Recall (macro): 0.8285939968404423 0.8355438029337897 0.8351158328706709 0.8400526010564672
Test metrics: {'accuracy': 0.8285939968404423, 'f1_macro': 0.8355438029337897, 'precision': 0.8351158328706709, 'recall': 0.8400526010564672}
